Motivation

The goal of this section is to identify frequent item associations in customer orders using the Apriori algorithm to generate rules of the form {A} → {B}. While the Apriori algorithm is widely used for association rule mining, it is not part of standard machine learning libraries like scikit-learn, and existing implementations often face scalability issues with large datasets.

The Instacart dataset contains over 32 million records, 3.2 million unique orders, and around 50,000 unique items. Traditional methods like pandas.crosstab() or itertools.combinations() struggle to handle such scale due to high memory usage, often leading to kernel crashes on machines with limited RAM.

To efficiently analyze pairwise item associations (itemsets of size 2), this implementation leverages Python generators, which yield one item pair at a time rather than storing all combinations in memory. This approach significantly reduces memory usage and enables scalable association analysis even on resource-constrained environments.

Apriori Algorithm

Apriori is an algorithm used to identify frequent item sets (in our case, item pairs). It does so using a "bottom up" approach, first identifying individual items that satisfy a minimum occurence threshold. It then extends the item set, adding one item at a time and checking if the resulting item set still satisfies the specified threshold. The algorithm stops when there are no more items to add that meet the minimum occurrence requirement.

Association Rules Mining

Once the item sets have been generated using apriori, we can start mining association rules. Given that we are only looking at item sets of size 2, the association rules we will generate will be of the form {A} -> {B}. One common application of these rules is in the domain of recommender systems, where customers who purchased item A are recommended item B.

3 key metrics to consider when evaluating association rules:

1. Support

This is the percentage of orders that contains the item set.

2. Confidence

Given two items, A and B, confidence measures the percentage of times that item B is purchased, given that item A was purchased. This is expressed as:

confidence{A->B} = support{A,B} / support{A}

3. Lift

Given two items, A and B, lift indicates whether there is a relationship between A and B, or whether the two items are occuring together in the same orders simply by chance. Unlike the confidence metric whose value may vary depending on direction (eg: confidence{A->B} may be different from confidence{B->A}), lift has no direction. This means that the lift{A,B} is always equal to the lift{B,A}:

lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B})

In summary, lift can take on the following values:

lift = 1 implies no relationship between A and B. (ie: A and B occur together only by chance)

lift > 1 implies that there is a positive relationship between A and B. (ie: A and B occur together more often than random)

lift < 1 implies that there is a negative relationship between A and B. (ie: A and B occur together less often than random)

In [6]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display

In [7]:
# Function that returns the size of an object in MB
def size(obj):
    return "{0:.2f} MB".format(sys.getsizeof(obj) / (1000 * 1000))

Data Preparation

In [8]:
orders = pd.read_csv('order_products__prior.csv')
print('orders -- dimensions: {0};   size: {1}'.format(orders.shape, size(orders)))
display(orders.head())

orders -- dimensions: (32434489, 4);   size: 1037.90 MB


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


Convert order data into format expected by the association rules function

In [9]:
# Convert from DataFrame to a Series, with order_id as index and item_id as value
orders = orders.set_index('order_id')['product_id'].rename('item_id')
display(orders.head(10))
type(orders)


order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
Name: item_id, dtype: int64

pandas.core.series.Series

Display summary statistics for order data

In [10]:
print('dimensions: {0};   size: {1};   unique_orders: {2};   unique_items: {3}'
      .format(orders.shape, size(orders), len(orders.index.unique()), len(orders.value_counts())))

dimensions: (32434489,);   size: 518.95 MB;   unique_orders: 3214874;   unique_items: 49677


Association Rules Function

In [ ]:
from collections import Counter, defaultdict
from itertools import combinations, groupby
import pandas as pd

# Defining helper functions
def freq(generator):
    counts = defaultdict(int)
    for item in generator:
        counts[item] += 1
    return pd.Series(counts).rename("freq")

def order_count(order_item):
    return len(set(order_item.index))

def get_item_pairs(order_item):
    order_item = order_item.reset_index().values
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
        for item_pair in combinations(item_list, 2):
            yield tuple(sorted(item_pair))  # sort to ensure (A,B) = (B,A)

def merge_item_stats(item_pairs, item_stats):
    return (item_pairs
                .merge(item_stats.rename(columns={'freq': 'freqA', 'support': 'supportA'}), left_on='item_A', right_index=True)
                .merge(item_stats.rename(columns={'freq': 'freqB', 'support': 'supportB'}), left_on='item_B', right_index=True))

def merge_item_name(rules, item_name):
    columns = ['itemA','itemB','freqAB','supportAB','freqA','supportA','freqB','supportB', 
               'confidenceAtoB','confidenceBtoA','lift']
    rules = (rules
                .merge(item_name.rename(columns={'item_name': 'itemA'}), left_on='item_A', right_on='item_id')
                .merge(item_name.rename(columns={'item_name': 'itemB'}), left_on='item_B', right_on='item_id'))
    return rules[columns]

# Load and convert orders to expected format (order_id as index, product_id as values)
orders_df = pd.read_csv('orders.csv')
prior = pd.read_csv('order_products__prior.csv')
orders = prior.merge(orders_df[['order_id']], on='order_id')
orders = orders.set_index('order_id')['product_id'].rename('item_id')  # Series format

# Association rule mining function
def association_rules(order_item, min_support):

    print("Starting order_item: {:22d}".format(len(order_item)))

    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100

    qualifying_items = item_stats[item_stats['support'] >= min_support].index
    order_item = order_item[order_item.isin(qualifying_items)]

    print("Items with support >= {}: {:15d}".format(min_support, len(qualifying_items)))
    print("Remaining order_item: {:21d}".format(len(order_item)))

    order_size = freq(order_item.index)
    qualifying_orders = order_size[order_size >= 2].index
    order_item = order_item[order_item.index.isin(qualifying_orders)]

    print("Remaining orders with 2+ items: {:11d}".format(len(qualifying_orders)))
    print("Remaining order_item: {:21d}".format(len(order_item)))

    item_stats = freq(order_item).to_frame("freq")
    item_stats['support'] = item_stats['freq'] / order_count(order_item) * 100

    # Reduce to most frequent items to save memory
    top_items = item_stats.sort_values("support", ascending=False).head(1000).index
    order_item = order_item[order_item.isin(top_items)]

    item_pair_gen = get_item_pairs(order_item)

    item_pairs = freq(item_pair_gen).to_frame("freqAB")
    item_pairs['supportAB'] = item_pairs['freqAB'] / len(qualifying_orders) * 100

    print("Item pairs: {:31d}".format(len(item_pairs)))

    item_pairs = item_pairs[item_pairs['supportAB'] >= min_support]

    print("Item pairs with support >= {}: {:10d}\n".format(min_support, len(item_pairs)))

    item_pairs.index = pd.MultiIndex.from_tuples(item_pairs.index, names=['item_A', 'item_B'])
    item_pairs = item_pairs.reset_index()

    item_pairs = merge_item_stats(item_pairs, item_stats)

    item_pairs['confidenceAtoB'] = item_pairs['supportAB'] / item_pairs['supportA']
    item_pairs['confidenceBtoA'] = item_pairs['supportAB'] / item_pairs['supportB']
    item_pairs['lift'] = item_pairs['supportAB'] / (item_pairs['supportA'] * item_pairs['supportB'])

    return item_pairs.sort_values('lift', ascending=False)


Association Rule Mining

In [26]:
# Run association rule mining
import time

start = time.time()
rules = association_rules(orders, min_support=0.01)
end = time.time()
print(f"Execution time: {end - start:.2f} seconds")

# Load item names and merge with rules
item_name = pd.read_csv('products.csv')
item_name = item_name.rename(columns={'product_id': 'item_id', 'product_name': 'item_name'})

rules_final = merge_item_name(rules, item_name)
display(rules_final.head(10))

Starting order_item:               32434489
Items with support >= 0.01:           10906
Remaining order_item:              29843570
Remaining orders with 2+ items:     3013325
Remaining order_item:              29662716
Item pairs:                          477048
Item pairs with support >= 0.01:      49528

Execution time: 177.66 seconds


,itemA,itemB,freqAB,supportAB,freqA,supportA,freqB,supportB,confidenceAtoB,confidenceBtoA,lift
0,Yotoddler Organic Pear Spinach Mango Yogurt,Organic Whole Milk Strawberry Beet Berry Yogur...,2823,0.093684,6121,0.203131,6272,0.208142,0.461199,0.450096,2.215789
1,Vanilla Almond Milk Yogurt,Almond Milk Strawberry Yogurt,1811,0.060100,5436,0.180399,5712,0.189558,0.333149,0.317052,1.757506
2,Blueberry on the Bottom Nonfat Greek Yogurt,Strawberry on the Bottom Nonfat Greek Yogurt,2767,0.091825,6370,0.211394,7625,0.253043,0.434380,0.362885,1.716627
3,Organic Greek Nonfat Yogurt With Mixed Berries,Organic Nonfat Greek Yogurt With Peaches,2208,0.073275,6888,0.228585,6063,0.201206,0.320557,0.364176,1.593178
4,Cherry Pie Fruit & Nut Bar,Apple Pie Fruit & Nut Food Bar,2002,0.066438,5536,0.183717,7469,0.247866,0.361633,0.268041,1.458987
5,Strawberry Rhubarb Yogurt,Mixed Berries Whole Milk Icelandic Style Skyr ...,2292,0.076062,6562,0.217766,7214,0.239403,0.349284,0.317716,1.458976
6,Blueberry on the Bottom Nonfat Greek Yogurt,Peach on the Bottom Nonfat Greek Yogurt,2371,0.078684,6370,0.211394,7768,0.257788,0.372214,0.305227,1.443873
7,Peach on the Bottom Nonfat Greek Yogurt,Strawberry on the Bottom Nonfat Greek Yogurt,2665,0.088441,7768,0.257788,7625,0.253043,0.343074,0.349508,1.355795
8,Broccoli & Apple Stage 2 Baby Food,Spinach Peas & Pear Stage 2 Baby Food,2339,0.077622,6830,0.226660,8030,0.266483,0.342460,0.291283,1.285109
9,Blueberry Muffin Bar,Apple Pie Fruit & Nut Food Bar,1826,0.060598,5749,0.190786,7469,0.247866,0.317620,0.244477,1.281421


Conclusion

From the output above, we see that the top associations are not surprising, with one flavor of an item being purchased with another flavor from the same item family. As mentioned, one common application of association rules mining is in the domain of recommender systems. Once item pairs have been identified as having positive relationship, recommendations can be made to customers in order to increase sales. And hopefully, along the way, also introduce customers to items they never would have tried before or even imagined existed!